Time-invariant

-- does `xarray` hand

In [1]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
# PO4 from NOAA_GFDL
po4 = xr.open_dataset('/global/cscratch1/sd/cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/po4/gr/v20180701/po4_Omon_GFDL-ESM4_historical_r1i1p1f1_gr_199001-200912.nc')['po4'].isel(lev=0)
no3 = xr.open_dataset('/global/cscratch1/sd/cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/no3/gr/v20180701/no3_Omon_GFDL-ESM4_historical_r1i1p1f1_gr_199001-200912.nc')['no3'].isel(lev=0)
ds = xr.merge([po4, no3])

In [ ]:
ds_mean = ds.mean('time')
ds_mean = ds_mean.stack(grid=['lat', 'lon'])
ds_mean = ds_mean.dropna('grid')
df = ds_mean.to_array().to_pandas().T

In [2]:
kmeans = KMeans(n_clusters=15, init='k-means++').fit(df.values)

NameError: name 'df' is not defined

In [ ]:
labels = kmeans.labels_ + 1

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))
p = ax.scatter(ds_mean.lon, ds_mean.lat, c=labels, vmin=1, vmax=15,
               cmap='Set2', zorder=0)
plt.colorbar(p)
ax.coastlines()

Time-varying

In [ ]:
ds_seas = ds.groupby('time.season').mean('time')

One season at a time

In [ ]:
kmeans_result = []
ds_seas = ds_seas.stack(grid=['lat', 'lon'])
ds_seas = ds_seas.dropna('grid')
for s in range(4):
    temp_df = ds_seas.isel(season=s).to_array().to_pandas().T
    kmeans = KMeans(n_clusters=15, init='k-means++').fit(temp_df.values)
    kmeans_result.append(kmeans)

In [ ]:
f, axs = plt.subplots(nrows=2, ncols=2, subplot_kw=dict(projection=ccrs.Robinson()),
                      figsize=(12, 12))
for s, ax in enumerate(axs.ravel()):
    result = kmeans_result[s]
    labels = result.labels_ + 1
    ax.scatter(ds_seas.lon, ds_seas.lat, c=labels, vmin=1, vmax=15,
               s=0.5)

All at once

In [ ]:
df1 = pd.DataFrame(np.array(ds_seas.po4.T), columns='po4_' + ds_seas.season.values)
df2 = pd.DataFrame(np.array(ds_seas.no3.T), columns='no3_' + ds_seas.season.values)
df = pd.concat([df1, df2], axis=1)

In [ ]:
kmeans = KMeans(n_clusters=15, init='k-means++').fit(df.values)
labels = kmeans.labels_ + 1

In [ ]:
ds_seas['labels'] = ('grid', labels)
ds_seas_unstack = ds_seas.unstack('grid')

In [ ]:
ds_seas_unstack.labels.plot(cmap=cmap)